In [1]:
import requests
import numpy as np
import pandas as pd

In [2]:
import base64
def get_token(client_id, client_secret):
    encoded = base64.b64encode(bytes(client_id+':'+client_secret, 'utf-8'))
    params = {'grant_type':'client_credentials'}
    header={'Authorization': 'Basic ' + str(encoded, 'utf-8')}
    r = requests.post('https://accounts.spotify.com/api/token', headers=header, data=params)
    if r.status_code != 200:
        print('Error en la request.', r.json())
        return None
    print('Token válido por {} segundos.'.format(r.json()['expires_in']))
    return r.json()['access_token']

def search_artist_id(string, token):
    ep_busqueda = 'https://api.spotify.com/v1/search'
    header = {'Authorization': f'Bearer {token}'}
    search_params = {'q': string, 'type': 'artist'}#, 'market': 'AR'}
    r = requests.get(ep_busqueda, headers=header, params=search_params)
    if r.status_code == 200:
        if len(r.json()['artists']['items']) > 0:
            return tuple(pd.DataFrame(r.json()['artists']['items']).sort_values(by='popularity', ascending=False).loc[0,['name', 'id']].values)
        else:
            print('Artist not found:', string)
            return (string, None)

    else:
        print("Error during the request")
        print(r.status_code)
        print(r.text)
        
        
def get_artist_info(artist_id, token):
    header = {'Authorization': f'Bearer {token}'}
    r = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}', headers=header)
    if r.status_code == 200:
        return r.json()
    else:
        print("Error during the request")
        print(r.status_code)
        print(r.text)
        
def get_related_artists(artist_id, token):
    header = {'Authorization': f'Bearer {token}'}
    r = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}/related-artists', headers=header)
    if r.status_code == 200:
        return [(artist['name'], artist['genres'], artist['id']) for artist in r.json()['artists']]
    else:
        print("Error during the request")
        print(r.status_code)
        print(r.text)

In [25]:
client_id = None
client_secret = None
with open('credentials.txt', 'r') as file:
    for line in file:
        if 'client_id' in line:
            start = line.find('\'')+1
            end = line[start:].find('\'')
            client_id = line[start:start+end]
            print('Found Client ID:', client_id)
        elif 'client_secret' in line:
            start = line.find('\'')+1
            end = line[start:].find('\'')
            client_secret = line[start:start+end]
            print('Found Client secret:', client_secret)
        if client_id != None and client_secret != None:
            break
    file.close()
    if client_id == None:
        print('Client ID not found in credentials.txt')
    if client_secret == None:
        print('Client secret not found in credentials.txt')

Found Client ID: 44b7b36ec145467f9a9eeaf7e417cf8b
Found Client secret: 7b4aa7a0ef4844048ab1d22430a1eb1f


In [26]:
token = get_token(client_id, client_secret)

Token válido por 3600 segundos.


In [6]:
lineup_day_1 = 'Travis Scott – Martin Garrix – Los Fabulosos Cadillacs – Brockhampton – DUKI – Madeon – Rita Ora – A Day to Remember – King Princess – LP – WOS – Denzel Curry – Yungblud – Nathy Peluso – J mena – AJR – Louta – Two Feet – Bizarrap – Fuego – La Delio Valdez – Dani – Kaydy Cain – Maye – Boombox Cartel – Cimafunk – Ms Nina – Feli Colina – Axel Fiks – Lucia Tacchetti – DJ Sky – Alejo y Valentin'
lineup_day_2 = 'The Strokes – Gwen Stefani – Armin Van Buuren – Vampire Weekend – Ratones Paranoicos – ILLENIUM – Kacey Musgraves – Litto Nebbia – Charli XCX – Jaden Smith – R3HAB – Hayley Kiyoko – El Mató a un Policía Motorizado – Rels B – Emmanuel Horvilleur – Kali Uchis – Paloma Mami – Fabiana Cantilo – Trueno – Amaia – Wallows – Yung Beef – Zoe Gotusso – Girl Ultra – Las Ligas Menores – Ainda – Ghetto Kids – D3FAI – Paco Leiva – LIMON – Louly'
lineup_day_3 = 'Guns N’ Roses – Lana del Rey – Cage the Elephant – James Blake – Alan Walker – Rezz – The Lumineers – Rex Orange County – MIKA – Perry Farrell’s Kind Heaven Orchestra – Airbag – Lauv – YSY A – Chris Lake – Pabllo Vittar – San Holo – Emilia – Idles – Masego – The Hu – Elsa y Elmar – Natalie Perez – Goldfish – Florian – Miranda Johansen – DABOW – El Buen Salvaje – Metro Live – Reydel'

In [7]:
str_to_list = lambda x: [band.strip() for band in x.split('–')]

In [8]:
lineup_day_1 = str_to_list(lineup_day_1)
lineup_day_2 = str_to_list(lineup_day_2)
lineup_day_3 = str_to_list(lineup_day_3)

In [9]:
artists_ids = [search_artist_id(artist, token) for artist in lineup_day_1+lineup_day_2+lineup_day_3]

Artist not found: Perry Farrell’s Kind Heaven Orchestra
Artist not found: Metro Live


In [10]:
artists_ids = {artist_name: artist_id for (artist_name, artist_id) in artists_ids}

In [11]:
search_artist_id('perry farrell', token)

('Perry Farrell', '2DS5RPK5A2GQTucYlgsQdE')

In [12]:
artists_ids['Perry Farrell’s Kind Heaven Orchestra'] = search_artist_id('perry farrell', token)[1]

In [13]:
artists_ids.pop('Metro Live')

In [14]:
get_artist_info(artists_ids[lineup_day_1[0]], token)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/0Y5tJX1MQlPlqiwlOH1tJY'},
 'followers': {'href': None, 'total': 7653146},
 'genres': ['rap'],
 'href': 'https://api.spotify.com/v1/artists/0Y5tJX1MQlPlqiwlOH1tJY',
 'id': '0Y5tJX1MQlPlqiwlOH1tJY',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/fec51156ed94b7ee30fbd4c7fd29b4a840e7daec',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/67497c55a766af3f57860c88562314a313fc137c',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/d7a58203669d00936dd5b89ba6f6e192c119532d',
   'width': 160}],
 'name': 'Travis Scott',
 'popularity': 98,
 'type': 'artist',
 'uri': 'spotify:artist:0Y5tJX1MQlPlqiwlOH1tJY'}

In [15]:
get_related_artists(artists_ids[lineup_day_2[5]], token)

[('William Black',
  ['bass trap', 'edm', 'pop edm', 'traprun'],
  '7d5SfGXKpgS3JK8BFIq59h'),
 ('ARMNHMR',
  ['brostep', 'edm', 'electro house', 'electronic trap', 'pop edm'],
  '0P2bZXPyjHYRW4guHVAFl1'),
 ('Nurko', ['bass trap', 'pop edm', 'traprun'], '757FXqX0Osk2pqtgv4E5v4'),
 ('Dabin',
  ['canadian electronic', 'catstep', 'chillstep', 'edm', 'pop edm'],
  '7lZauDnRoAC3kmaYae2opv'),
 ('Said the Sky', ['edm', 'electropop', 'pop edm'], '4LZ4De2MoO3lP6QaNCfvcu'),
 ('Slushii',
  ['bass trap',
   'brostep',
   'edm',
   'electro house',
   'electronic trap',
   'pop edm'],
  '41rVuRHYAiH7ltBTHVR9We'),
 ('Seven Lions',
  ['brostep', 'edm', 'electro house', 'pop edm'],
  '6fcTRFpz0yH79qSKfof7lp'),
 ('Skrux',
  ['catstep', 'chillstep', 'edm', 'pop edm'],
  '6cwRqYSDYwWsLveCcDRhyY'),
 ('Kasbo',
  ['catstep',
   'edm',
   'electropop',
   'indie electropop',
   'indie poptimism',
   'indietronica',
   'new french touch',
   'pop edm',
   'swedish electropop',
   'tropical house',
   'vapor so